# **Economic Freedom and Happiness Across Europe**

In [1]:
import pandas as pd
import plotly.io as pio
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pycountry 

import subprocess
import sys

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

# Function for converting to iso 
def getIsoCode(country_name):
    try:
        country_iso = pycountry.countries.search_fuzzy(country_name)[0]
        return country_iso.alpha_3
    except LookupError:
        return ""

In [2]:
%%html
<style>
.cartesianlayer .bars path,
.colorbar .cbfills,
.slider-rail-rect { clip-path: inset(0% 0% 0% 0% round 6px); }

.legendundefined { clip-path: inset(0% 0% 0% 0% round 3px); }

.gtitle .line:nth-of-type(2) { fill: grey; }

.output_area:last-of-type,
.cell_output {
    filter: drop-shadow(0px 0px 10px rgba(0, 0, 0, 0.2));
    -webkit-backdrop-filter: blur(0); // fixes a safari drop-shadow bug
}

#notebook-container img,
main img {
    box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.2);
}

#notebook-container img,
main img,
.plotly-graph-div {
    border-radius: 10px;
    overflow: hidden;
    border: 1px solid black;
}
</style>

In [3]:
happiness_data = pd.read_csv("happiness.csv")
happiness_data["Country"] = happiness_data["Country or region"]
happiness_data = happiness_data.drop(columns=["Country or region", "Overall rank"])
happiness_data = happiness_data.rename(columns={"Score": "Happiness Score"})

In [4]:
freedom_data = pd.read_csv("freedom.csv", low_memory=False)
# Normalize country column
freedom_data = freedom_data.rename(columns={"countries": "Country"})

# Filter out years other than 2019
freedom_data["year"] = pd.to_numeric(freedom_data["year"], errors='coerce')
freedom_data = freedom_data[freedom_data["year"] == 2019]

# Reset and Drop
freedom_data = freedom_data.reset_index()
freedom_data = freedom_data.drop(columns="index")

freedom_filtered = freedom_data[freedom_data['Country'].isin([
    "Albania", "Andorra", "Austria", "Belarus", "Belgium", 
    "Bosnia and Herzegovina", "Bulgaria", "Croatia", "Cyprus", "Czech Republic", 
    "Denmark", "Estonia", "Finland", "France", "Germany", "Greece", 
    "Hungary", "Iceland", "Ireland", "Italy", "Kosovo", "Latvia", 
    "Liechtenstein", "Lithuania", "Luxembourg", "Malta", "Moldova", "Monaco", 
    "Montenegro", "Netherlands", "North Macedonia", "Norway", "Poland", "Portugal", 
    "Romania", "Russia", "San Marino", "Serbia", "Slovakia", "Slovenia", "Spain", 
    "Sweden", "Switzerland", "Ukraine", "United Kingdom", "Vatican City"
])]

columns_to_keep = [
    'Country', 'ef_government', 'ef_legal', 'ef_money', 'ef_trade', 'ef_regulation', 'ef_rank'
]

freedom_data = freedom_filtered[columns_to_keep]

freedom_data = freedom_data.rename(columns={
    "ef_government": "Size of Government",
    "ef_legal": "Legal System and Property Rights",
    "ef_money": "Access to Sound Money",
    "ef_trade": "Freedom to Trade Internationally",
    "ef_regulation": "Regulation of Credit, Labor, and Business",
    "ef_rank": "Economic Freedom Rank"
})



In [5]:
data = pd.merge(happiness_data, freedom_data, on='Country')
data = data.drop_duplicates(subset=['Country'])

data["iso"] = data['Country'].apply(getIsoCode)

# CREATING NEW RANK
sorted_df = data.sort_values(by='Economic Freedom Rank')
# Display the sorted DataFrame with ranks
data = sorted_df

# Introduction

In today’s free market economies of Europe it is important to have the appropriate legal systems in place to protect the rights of people and businesses alike. In western Europe these have had more time to slowly grow and improve whilst the former East Bloc has had way less time to develop since they were opened to capitalism almost 35 years ago. Studies show that the more legal protection people experience, the happier they are (D’Ambrioso et al., 2020). Others claim the happiness of people can be attributed to other factors. These factors, such as culture, region and climate would explain the different levels of happiness without having to mention legal protection. The stark difference between these perspectives is what we will explore in this data story.

To explore this contrast we will use 2 different datasets. One contains data about the happiness index of each country and the other contains the freedom indices. For this data story we will be looking at the data from 2019 and focused on Europe. We opted for this because Europe is unique in the way we have eastern countries who just recently opened their markets and are still in development while also having western countries who are fully developed over centuries. Another advantage is we share somewhat similar cultures and values so external factors are minimal.

## Analysis Across Countries

In [6]:
top_5 = data.nlargest(5, 'Happiness Score')
bottom_5 = data.nsmallest(5, 'Happiness Score')

# Create trace for top 5 countries
trace_top = go.Bar(
    x=top_5['Country'],
    y=top_5['Happiness Score'],
    name='Top 5 Happiest Countries',
    marker=dict(color='#049b9e')
)

# Create trace for bottom 5 countries
trace_bottom = go.Bar(
    x=bottom_5['Country'],
    y=bottom_5['Happiness Score'],
    name='Bottom 5 Happiest Countries',
    marker=dict(color='#c85fac')
)

# Define layout
layout = go.Layout(
    title='<b>Highest and Lowest Happiness Scores</b>',
    xaxis=dict(title='Country'),
    yaxis=dict(title='Happiness Score', range=[0, 10]),  # Set the same y-axis range for both plots
    barmode='group',  # Display bars side by side
    bargap=0.2,  # Gap between bars
    bargroupgap=0.1,  # Gap between groups of bars
    width=750,
    height=500,
)

#c85fac, #049b9e
# Create figure object
fig = go.Figure(data=[trace_top, trace_bottom], layout=layout)
fig.update_layout(title_x=0.5)
# Show the plot

fig.show()

In [7]:
import plotly.express as px

# Merge data with latitude and longitude
custom_colors = [
    [0.0, "#ffffff"],
    [1.0, "#c85fac"],
]

# Create the choropleth map focusing on Europe
fig = px.choropleth(data, locations="iso",
                    title='<b>Economic Freedom In Europe</b>',
                    locationmode='ISO-3',  # Use ISO-3 country codes
                    color="Economic Freedom Rank",
                    hover_name="iso",
                    width=750,
                    height=500,
                    hover_data=['iso', 'Economic Freedom Rank'],
                    color_continuous_scale=custom_colors,  # Adjust color scale as needed
                    scope='europe',  # Focus on Europe
                    center={"lat": 51.1657, "lon": 10.4515},  # Center of Europe (approximate)
                    projection='natural earth',
                    labels={'Economic Freedom Rank': 'Economic Freedom Rank', 'iso': 'Country Code'},
                    )

fig.update_layout(title_x=0.5, margin=dict(b=0, l=0))

# Show the plot
fig.show()


### Correlation Matrix of Freedom, Happiness, and Health Indicators

The relationship between economic freedom, happiness, and health indicators is complex and multifaceted. To unravel this intricate web, we examine a correlation matrix featuring various measures of economic freedom and happiness/health indicators. This matrix includes GDP per capita, social support, healthy life expectancy, freedom to make life choices, generosity, perceptions of corruption, and several aspects of economic freedom such as government size, gender legal rights, legal system and property rights, access to sound money, freedom to trade internationally, and regulation.

In the correlation matrix, the strength and direction of the correlation are visually represented by a color scale ranging from -1 (indicating a strong negative correlation) to +1 (indicating a strong positive correlation). Shades of pink and purple represent positive correlations, while shades of teal indicate negative correlations.


In [8]:
columns_to_keep=['GDP per capita', 'Social support',
       'Healthy life expectancy', 'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption', 'Size of Government',
       'Legal System and Property Rights', 'Access to Sound Money', 'Freedom to Trade Internationally', 'Regulation of Credit, Labor, and Business']

corr = data[columns_to_keep].corr().round(1)
color_continuous_scale='tropic'

fig = px.imshow(corr,
                x=corr.index,
                y=corr.columns,
                color_continuous_scale='tropic',
                zmin=-1, zmax=1,
                labels=dict(color='Correlation'),
                text_auto=True)

fig.update_layout(title='<b>Correlation Matrix of Economic Freedom and Happiness Indicators</b>',
                  xaxis_title='',
                  yaxis_title='',
                  title_x=0.5,
                  coloraxis_colorbar=dict(title='Correlation'),
                  font=dict(family='Arial', size=12),
                  width=750, height=600,
                  margin=dict(b=10, r=10)),
                     

fig.update_traces(textfont_size=10, hoverinfo='text')
fig.show()



### Key Insights from the Correlation Matrix

One of the standout findings is the significant role of Economic Freedom - Legal System and Property Rights. This indicator shows strong positive correlations with various measures of societal well-being:

- *GDP per capita*
- *Social support*
- *Healthy life expectancy*
- *Freedom to make life choices*

These relationships are evident from the darker shades of pink and purple at these intersections in the matrix. Furthermore, Economic Freedom - Legal System and Property Rights is positively correlated with perceptions of corruption, suggesting that stronger legal freedoms might be associated with lower levels of perceived corruption.

### Zooming In: Happiness and Legal Freedom

To further explore the relationship between happiness and legal freedom, we delve into a scatter plot focusing on the correlation between the Happiness Score and Economic Freedom - Legal System and Property Rights. The scatter plot reveals a clear linear relationship, underscoring that higher legal freedom is associated with higher happiness levels. The trend line in the scatter plot reinforces this observation, illustrating that as legal freedom increases, overall happiness scores tend to rise.

In [9]:
# Convert data to numpy arrays
X = np.array(data['Legal System and Property Rights'])  # Independent variable
y = np.array(data['Happiness Score'])  # Dependent variable

# Compute coefficients of linear regression (y = mx + c)
X_mean = np.mean(X)
y_mean = np.mean(y)
m = np.sum((X - X_mean) * (y - y_mean)) / np.sum((X - X_mean) ** 2)
c = y_mean - m * X_mean

# Predicted values of y based on the regression equation
y_pred = m * X + c

# Plotting the scatter plot and regression line using Plotly
fig = go.Figure()

# Scatter plot of data points
fig.add_trace(go.Scatter(
    x=X,
    y=y,
    mode='markers',
    marker=dict(color='#c85fac'),
    name='Data Points'
))

# Regression line
fig.add_trace(go.Scatter(
    x=X,
    y=y_pred,
    mode='lines',
    line=dict(color='black', width=2),
    name='Regression Line'
))

# Layout settings with adjusted axis ranges
fig.update_layout(
    title='<b>Legal System and Property Rights vs. Happiness Score</b>',
    xaxis_title='Legal System and Property Rights',
    yaxis_title='Happiness Score',
    showlegend=False,
    hovermode='closest',
    template='plotly_white',
    width=750,
    height=500,

    yaxis_range=[0, 10],  # Set range for y-axis
)

# Show the plot
fig.update_layout(title_x=0.5)
fig.show()

### Historical Context: Former Soviet Union vs. Not Former Soviet Union

Another layer of analysis compares happiness levels between countries that were part of the former Soviet Union and those that were not. A box plot provides a visual comparison:

- *Former Soviet Union countries* tend to have lower overall happiness scores.
- *Non-Former Soviet Union countries* exhibit higher and more varied happiness scores.

This comparison highlights the significant impact of historical and political contexts on current happiness levels. The generally lower happiness levels in former Soviet Union countries suggest that the legacy of restricted freedoms continues to influence societal well-being.
The analysis clearly indicates that legal freedom, particularly through Economic Freedom - Legal System and Property Rights, is a critical factor for societal well-being and happiness. The strong correlations with various indicators and the positive relationship with overall happiness underscore the importance of a robust legal system and property rights in enhancing societal well-being.
Additionally, the comparison between former Soviet Union countries and those not part of it reveals that historical and political contexts play a significant role in determining current happiness levels. This analysis suggests that policies aimed at improving legal systems and property rights could have a substantial positive impact on overall societal well-being, especially in regions with a legacy of restricted freedoms.
In summary, fostering legal freedom and strengthening property rights are essential for promoting happiness and health within societies, and historical contexts must be considered in policy-making to address the nuanced needs of different regions.

In [10]:
import plotly.graph_objects as go

ex_soviet_states = ["Belarus", "Estonia", "Latvia", "Lithuania", "Moldova", "Russia", "Ukraine", "Bosnia and Herzegovina", "Albania"]

data['Ex-Soviet State'] = data['Country'].isin(ex_soviet_states)
data['Ex-Soviet State'] = data['Ex-Soviet State'].apply(lambda x: 'True' if x else 'False')

ex = data[data['Ex-Soviet State'] == "True"]
non_ex = data[data['Ex-Soviet State'] == "False"]

fig = go.Figure()

fig.add_trace(go.Box(
    y=ex["Happiness Score"],
    name="Former Soviet Union",
    fillcolor="#049b9e", 
    line_color="black",
))

fig.add_trace(go.Box(
    y=non_ex["Happiness Score"],
    name="Not Former Soviet Union",
    fillcolor="#c85fac",  
    line_color="black",  
))

fig.update_layout(
    yaxis_title="Happiness Score",
    width=750,
    height=500,
    title_x=0.5
)

fig.show()

In [11]:
import pandas as pd
import plotly.graph_objects as go


columns_to_keep = ['Size of Government', 'Legal System and Property Rights', 
                   'Access to Sound Money', 'Freedom to Trade Internationally', 
                   'Regulation of Credit, Labor, and Business']

# Filter the DataFrames to keep only the specified columns
filtered_data1 = ex[columns_to_keep]
filtered_data2 = non_ex[columns_to_keep]

# Calculate the average values for each column in the datasets
average_data1 = filtered_data1.mean()
average_data2 = filtered_data2.mean()

# Create the initial bar plot for the first dataset
fig = go.Figure()

fig.add_trace(go.Bar(
    x=average_data1.index,
    y=average_data1.values,
    name='Ex-Soviet',
    marker=dict(color='#049b9e')
))

# Add the second dataset, but initially make it invisible
fig.add_trace(go.Bar(
    x=average_data2.index,
    y=average_data2.values,
    name='Non Ex-Soviet',
    visible=False,
    marker=dict(color='#c85fac')
))

# Create the dropdown menu
fig.update_layout(

    updatemenus=[
        dict(
            buttons=[
                dict(
                    args=[{'visible': [True, False]}],
                    label='Ex Soviet',
                    method='update'
                ),
                dict(
                    args=[{'visible': [False, True]}],
                    label='Non Ex Soviet',
                    method='update'
                )
            ],
        direction='down',
            showactive=True,
            x=1.3,  # Position the menu to the right
            xanchor='right',
            y=1,  # Position it at the same level as the title
            yanchor='top'
        )
    ],


    width=750,
    height=500,
        title='<b>Average Values for Economic Freedom Dimensions</b>',
    xaxis_title='Economic Freedom Dimensions',
    yaxis_title='Average Values',
     yaxis=dict(range=[5, 10]),
title_x = 0.5
)


# Show the figure
fig.show()


Je ziet bij ex-soviet een grotere size of government, en alle andere waarde zijn lager.

Plot gaat van 5 to 10 ipv 0 tot 10 om de verschillen duidelijker zichtbaar te maken. Maar kan ook terug naar 0 tot 10

## Summary

## References

1. D’Ambrosio, C., Jäntti, M., & Lepinteur, A. (2019). Money and Happiness: Income, Wealth and Subjective Well-Being. Social Indicators Research, 148(1), 47–66. https://doi.org/10.1007/s11205-019-02186-w

## External Links